In [3]:
import os

os.environ["NOTION_DATABASE_TYPE"] = "book fragment"
os.environ["NOTION_DATABASE_ID"] = "e9a60798218a427cb9e351c78a360ae2"


In [1]:
import requests
import random
import time
import os

HEADERS = {
    "Authorization": f"Bearer {os.getenv('NOTION_SECRET_TOKEN')}",
    "Notion-version": "2021-05-13",
}

MAX_PAGINATION = 20


def retrieve_database(database_id, query=None) -> dict:
    database = {"has_more": True, "next_cursor": None, "results": []}

    i = 0
    while database["has_more"]:
        i += 1
        if i > MAX_PAGINATION:
            raise NotImplementedError(f"Max pagination reached ({MAX_PAGINATION})")

        next_cursor = database.get("next_cursor")
        if not next_cursor and i > 1:
            break

        next_database = _retrieve_paginated_database(database_id, query, next_cursor)
        for k, v in next_database.items():
            if k == "results":
                database["results"].extend(v)
            else:
                database[k] = v

        time.sleep(1)
    return database


def _retrieve_paginated_database(database_id, query=None, start_cursor=None) -> dict:
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    data = query or {}
    if start_cursor:
        data["start_cursor"] = start_cursor
    print(f"POST {url} with data: {data}")
    response = requests.post(url=url, headers=HEADERS, json=data)
    response.raise_for_status()
    database = response.json()
    return database


def retrieve_page_content(page_id) -> dict:
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    print(f"GET {url}")
    response = requests.get(url=url, headers=HEADERS)
    response.raise_for_status()
    page = response.json()
    return page


In [2]:
import re
import random
import json
import os

DB_TYPE = os.getenv("NOTION_DATABASE_TYPE")


def get_random_page(database):
    if database["has_more"]:
        raise NotImplementedError("Database has more pages. Must implement pagination.")

    idx = random.choice(list(range(len(database["results"]))))
    page = database["results"][idx]
    return page


def get_page_id(database_page):
    return database_page["id"]


def get_page_url(database_page):
    return database_page["url"]


def get_page_topic(database_page):
    return database_page["properties"]["Tags"]["multi_select"][0]["name"]


def get_page_name(database_page):
    return database_page["properties"].get("Name", {})["title"][0]["text"]["content"]


def parse_page_text_content(block_page):
    try:
        text = ""
        for r in block_page["results"]:
            result_type = r["type"]
            result_text_blocks = r[result_type]["text"]
            if not result_text_blocks:
                text += "<br>"
            else:
                for t in result_text_blocks:
                    content = t["text"]["content"]
                    content_is_bold = t.get("annotations", {}).get("bold")
                    if result_type.startswith("heading"):
                        styled_content = f"<h3>{content}</h3><br>"
                    elif content_is_bold:
                        styled_content = f"<b>{content}</b><br>"
                    else:
                        styled_content = f"{content}<br>"
                    text += styled_content

    except Exception as e:
        print(f"Error parsing text from page {json.dumps(block_page, indent=2)}")
        raise
    return dict(text=text)


def parse_page_quote_content(block_page):
    try:
        quote = block_page["results"][0]["paragraph"]["text"][0]["text"]["content"]
        attrib = block_page["results"][1]["paragraph"]["text"][0]["text"]["content"]
    except Exception as e:
        print(f"Error parsing quote from page {json.dumps(block_page, indent=2)}")
        raise
    return dict(
        quote=quote,
        attrib=attrib,
    )


def parse_page_recipe_content(block_page):
    recipe = parse_page_text_content(block_page)["text"]
    return dict(recipe=recipe)


def parse_page_book_fragment(block_page):
    text = parse_page_text_content(block_page)["text"]
    return dict(text=text)


content_parser = {
    "quote": parse_page_quote_content,
    "recipe": parse_page_recipe_content,
    "book fragment": parse_page_book_fragment,
}


def parse_page_content(block_page):
    return content_parser[DB_TYPE](block_page)


In [5]:
    database = retrieve_database(
        database_id=os.getenv("NOTION_DATABASE_ID"),
    )

POST https://api.notion.com/v1/databases/e9a60798218a427cb9e351c78a360ae2/query with data: {}


In [12]:
book_frag_page = get_random_page(database)
book_frag_page_id = get_page_id(book_frag_page)
book_frag_page_content = retrieve_page_content(book_frag_page_id)


GET https://api.notion.com/v1/blocks/93dc89e8-277b-4200-95d7-26a7796ae7e7/children


In [13]:
book_frag_page_content

{'object': 'list',
 'results': [{'object': 'block',
   'id': '5175e042-060a-4595-bd65-88d345080bbe',
   'parent': {'type': 'page_id',
    'page_id': '93dc89e8-277b-4200-95d7-26a7796ae7e7'},
   'created_time': '2023-09-03T13:24:00.000Z',
   'last_edited_time': '2023-09-03T13:24:00.000Z',
   'created_by': {'object': 'user',
    'id': '8f75565e-980a-4508-92c6-154a3fa6afd2'},
   'last_edited_by': {'object': 'user',
    'id': '8f75565e-980a-4508-92c6-154a3fa6afd2'},
   'has_children': False,
   'archived': False,
   'type': 'paragraph',
   'paragraph': {'color': 'default',
    'text': [{'type': 'text',
      'text': {'content': "Those who look down upon this world\nwill surely take hold and try to change things\nBut this is a plan\nI've always seen fail\nThe world is Tao's own vessel\nIt is perfection manifest\nIt cannot be changed\nIt cannot be improved\nFor those who go on tampering, it's ruined\nFor those who try to grasp, it's gone",
       'link': None},
      'annotations': {'bold': F

In [17]:

book_frag_page_content["results"][1]

{'object': 'block',
 'id': '27dcead3-cf50-4b47-95cb-ccfca32edb51',
 'parent': {'type': 'page_id',
  'page_id': '93dc89e8-277b-4200-95d7-26a7796ae7e7'},
 'created_time': '2023-09-03T13:24:00.000Z',
 'last_edited_time': '2023-09-03T13:24:00.000Z',
 'created_by': {'object': 'user',
  'id': '8f75565e-980a-4508-92c6-154a3fa6afd2'},
 'last_edited_by': {'object': 'user',
  'id': '8f75565e-980a-4508-92c6-154a3fa6afd2'},
 'has_children': False,
 'archived': False,
 'type': 'paragraph',
 'paragraph': {'color': 'default',
  'text': [{'type': 'text',
    'text': {'content': "Allow your life to unfold naturally\nKnow that it too is a vessel of perfection\nJust as you breathe in and breathe out\nSometimes you're ahead and other times behind\nSometimes you're strong and other times weak\nSometimes you're with people and other times alone",
     'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color'